## NoteBook with Purely the LSTM Model contained - For testing

In [30]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.base import clone
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from functools import reduce

# Import the DF's

In [31]:
forecastdemand_vic = pd.read_csv("../data/Australia/a/forecastdemand_vic.csv")
forecastdemand_sa = pd.read_csv("../data/Australia/b/forecastdemand_sa.csv")
forecastdemand_qld = pd.read_csv("../data/Australia/c/forecastdemand_qld.csv")
temperature_qld = pd.read_csv("../data/Australia/d/temprature_qld.csv")  # There's a typo in 'temperature' in the file path
temperature_sa = pd.read_csv("../data/Australia/d/temprature_sa.csv")  # Same typo as above
temperature_vic = pd.read_csv("../data/Australia/d/temprature_vic.csv")  # Same typo as above
totaldemand_qld = pd.read_csv("../data/Australia/d/totaldemand_qld.csv")
totaldemand_sa = pd.read_csv("../data/Australia/d/totaldemand_sa.csv")
totaldemand_vic = pd.read_csv("../data/Australia/d/totaldemand_vic.csv")

In [32]:
forecastdemand_qld['DATETIME'] = pd.to_datetime(forecastdemand_qld['DATETIME'], format="%Y-%m-%d %H:%M:%S")
temperature_qld['DATETIME'] = pd.to_datetime(temperature_qld['DATETIME'], format="%d/%m/%Y %H:%M")
totaldemand_qld['DATETIME'] = pd.to_datetime(totaldemand_qld['DATETIME'], format="%Y-%m-%d %H:%M:%S")



# Drop duplicates with latest last changed feature

In [33]:
forecastdemand_qld['LASTCHANGED'] = pd.to_datetime(forecastdemand_qld['LASTCHANGED'])
forecastdemand_qld.sort_values(by='LASTCHANGED', ascending=False, inplace=True)
forecastdemand_qld.drop_duplicates(subset='DATETIME', keep='first', inplace=True)
print(forecastdemand_qld.shape)


(73833, 6)


In [34]:
dataframes_qld = [forecastdemand_qld, totaldemand_qld, temperature_qld]

for df in dataframes_qld:
    df['YEAR'] = df['DATETIME'].dt.year
    df['MONTH'] = df['DATETIME'].dt.month
    df['DAY'] = df['DATETIME'].dt.day
    df['HOUR'] = df['DATETIME'].dt.hour
    df['MINUTE'] = df['DATETIME'].dt.minute

merge_keys = ['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE']
df_qld_merged = reduce(lambda left, right: pd.merge(left, right, on=merge_keys, how='inner'), dataframes_qld)

print(df_qld_merged.head())

   PREDISPATCHSEQNO REGIONID_x  PERIODID  FORECASTDEMAND         LASTCHANGED  \
0        2021031740       QLD1         1         5714.08 2021-03-17 23:31:33   
1        2021031739       QLD1         1         5924.06 2021-03-17 23:01:36   
2        2021031738       QLD1         1         6148.23 2021-03-17 22:31:36   
3        2021031737       QLD1         1         6309.79 2021-03-17 22:01:34   
4        2021031736       QLD1         1         6459.56 2021-03-17 21:31:31   

           DATETIME_x  YEAR  MONTH  DAY  HOUR  MINUTE          DATETIME_y  \
0 2021-03-18 00:00:00  2021      3   18     0       0 2021-03-18 00:00:00   
1 2021-03-17 23:30:00  2021      3   17    23      30 2021-03-17 23:30:00   
2 2021-03-17 23:00:00  2021      3   17    23       0 2021-03-17 23:00:00   
3 2021-03-17 22:30:00  2021      3   17    22      30 2021-03-17 22:30:00   
4 2021-03-17 22:00:00  2021      3   17    22       0 2021-03-17 22:00:00   

   TOTALDEMAND REGIONID_y                      LOCATION 

In [35]:
df_qld_merged.drop(columns=['REGIONID_x', 'DATETIME_x'], inplace=True)
print(df_qld_merged.columns)

Index(['PREDISPATCHSEQNO', 'PERIODID', 'FORECASTDEMAND', 'LASTCHANGED', 'YEAR',
       'MONTH', 'DAY', 'HOUR', 'MINUTE', 'DATETIME_y', 'TOTALDEMAND',
       'REGIONID_y', 'LOCATION', 'DATETIME', 'TEMPERATURE'],
      dtype='object')


# Add in engineered features - Weekend and season

In [36]:
df_qld_merged['is_weekend'] = df_qld_merged['DATETIME'].dt.dayofweek >= 5

df_qld_merged['season'] = df_qld_merged['DATETIME'].dt.month % 12 // 3
df_qld_merged['season'] = df_qld_merged['season'].map({0: 'Summer', 1: 'Autumn', 2: 'Winter', 3: 'Spring'}, na_action='ignore')


australian_seasons = {
    12: 'Summer', 1: 'Summer', 2: 'Summer',
    3: 'Autumn', 4: 'Autumn', 5: 'Autumn',
    6: 'Winter', 7: 'Winter', 8: 'Winter',
    9: 'Spring', 10: 'Spring', 11: 'Spring'
    }
df_qld_merged['season'] = df_qld_merged['MONTH'].map(australian_seasons)

# Add in a two more engineered features - Cooling and heating vectors 

In [37]:
df_qld_merged['Cooling'] = df_qld_merged['TEMPERATURE'].apply(lambda x: max(0, x - 24))

df_qld_merged['Heating'] = df_qld_merged['TEMPERATURE'].apply(lambda x: max(0, 20 - x))

print(df_qld_merged[['Cooling', 'Heating']].head())  

   Cooling  Heating
0      0.0      0.5
1      0.0      0.4
2      0.0      0.6
3      0.0      0.5
4      0.0      0.4


# Importing Public Holiday Data as another predictive Feature

In [39]:
file_path = '../data/public_holidays/Aus_public_hols_2009-2022-1.csv'
try:
    public_holidays = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    public_holidays = pd.read_csv(file_path, encoding='latin1')

In [40]:
public_holidays['Date'] = pd.to_datetime(public_holidays['Date'])
print(public_holidays.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1406 entries, 0 to 1405
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           1406 non-null   datetime64[ns]
 1   State          1406 non-null   object        
 2   Weekday        1406 non-null   object        
 3   Day_In_Lieu    1406 non-null   int64         
 4   Part_Day_From  1406 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 55.1+ KB
None


# Merging the public holiday data into the master QLD file (1 for public holiday, 0 for not)

In [41]:
public_holidays['Date'] = pd.to_datetime(public_holidays['Date'])
qld_holidays = public_holidays[public_holidays['State'] == 'QLD']
qld_holiday_dates = qld_holidays['Date'].dt.date.unique()
df_qld_merged['DATETIME'] = pd.to_datetime(df_qld_merged['DATETIME'])
df_qld_merged['Public_Holiday'] = df_qld_merged['DATETIME'].dt.date.isin(qld_holiday_dates).astype(int)
print(df_qld_merged.head())


   PREDISPATCHSEQNO  PERIODID  FORECASTDEMAND         LASTCHANGED  YEAR  \
0        2021031740         1         5714.08 2021-03-17 23:31:33  2021   
1        2021031739         1         5924.06 2021-03-17 23:01:36  2021   
2        2021031738         1         6148.23 2021-03-17 22:31:36  2021   
3        2021031737         1         6309.79 2021-03-17 22:01:34  2021   
4        2021031736         1         6459.56 2021-03-17 21:31:31  2021   

   MONTH  DAY  HOUR  MINUTE          DATETIME_y  TOTALDEMAND REGIONID_y  \
0      3   18     0       0 2021-03-18 00:00:00      5737.03       QLD1   
1      3   17    23      30 2021-03-17 23:30:00      5897.64       QLD1   
2      3   17    23       0 2021-03-17 23:00:00      6144.16       QLD1   
3      3   17    22      30 2021-03-17 22:30:00      6264.63       QLD1   
4      3   17    22       0 2021-03-17 22:00:00      6443.62       QLD1   

                       LOCATION            DATETIME  TEMPERATURE  is_weekend  \
0  Brisbane Archer

## Import the new DF that was created in the PV Scraper

In [48]:
solar_PV_production = pd.read_csv('../data/PV_Data/raw_data/unzipped_data/combined_df_grouped_sorted.csv')

In [49]:
qld_solar_df = solar_PV_production[solar_PV_production['State'] == 'QLD']

In [50]:
qld_solar_df['INTERVAL_DATETIME'] = pd.to_datetime(qld_solar_df['INTERVAL_DATETIME'], infer_datetime_format=True)

/var/folders/bv/yhyl185564n2bzh9449xdfjw0000gn/T/ipykernel_9542/730933358.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  qld_solar_df['INTERVAL_DATETIME'] = pd.to_datetime(qld_solar_df['INTERVAL_DATETIME'], infer_datetime_format=True)
/var/folders/bv/yhyl185564n2bzh9449xdfjw0000gn/T/ipykernel_9542/730933358.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qld_solar_df['INTERVAL_DATETIME'] = pd.to_datetime(qld_solar_df['INTERVAL_DATETIME'], infer_datetime_format=True)


In [51]:
print(qld_solar_df['INTERVAL_DATETIME'].dtype)

datetime64[ns]


## Merge with the Master QLD DF

In [52]:
complete_merged_df = df_qld_merged.merge(
    qld_solar_df[['INTERVAL_DATETIME', 'POWER']], 
    how='left', 
    left_on='DATETIME', 
    right_on='INTERVAL_DATETIME'
)

In [53]:
complete_merged_df.rename(columns={'POWER': 'Solar_Production'}, inplace=True)

## Drop any rows where we dont have Solar Prod Data

In [54]:
complete_merged_df = complete_merged_df.dropna(subset=['INTERVAL_DATETIME', 'Solar_Production'])

## Finally Create LSTM Model

In [55]:
numerical_features = ['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'TEMPERATURE', 'Heating', 'Cooling', 'Solar_Production']
categorical_features = ['season']
binary_features = ['is_weekend', 'Public_Holiday']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features),
        ('bin', 'passthrough', binary_features)
    ])

X = complete_merged_df.drop(['TOTALDEMAND'], axis=1)
y = complete_merged_df['TOTALDEMAND'].values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

import torch
from torch.utils.data import TensorDataset, DataLoader

X_train_tensor = torch.tensor(X_train.astype(np.float32))
y_train_tensor = torch.tensor(y_train.astype(np.float32))
X_test_tensor = torch.tensor(X_test.astype(np.float32))
y_test_tensor = torch.tensor(y_test.astype(np.float32))

train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 64  # You can adjust this size
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size)


In [59]:
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Here x should have dimensions [batch, 1, features] - adding sequence dimension
        x = x.unsqueeze(1)  
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)  
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

input_dim = X_train.shape[1]  # number of features
hidden_dim = 100  # can be changed
num_layers = 2  # number of LSTM layers
output_dim = 1  # one output
num_epochs = 50
model = LSTMModel(input_dim, hidden_dim, num_layers, output_dim)

In [57]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [60]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # If using GPU
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

Epoch 1, Loss: 32935598.0
Epoch 2, Loss: 22509648.0
Epoch 3, Loss: 20209946.0
Epoch 4, Loss: 19002858.0
Epoch 5, Loss: 16043596.0
Epoch 6, Loss: 12169731.0
Epoch 7, Loss: 8053302.0
Epoch 8, Loss: 5716660.5
Epoch 9, Loss: 7047811.5
Epoch 10, Loss: 2865069.75
Epoch 11, Loss: 3449994.0
Epoch 12, Loss: 896480.8125
Epoch 13, Loss: 2010407.625
Epoch 14, Loss: 929408.3125
Epoch 15, Loss: 1344079.625
Epoch 16, Loss: 1461755.375
Epoch 17, Loss: 1252422.5
Epoch 18, Loss: 616108.3125
Epoch 19, Loss: 1084384.375
Epoch 20, Loss: 829844.75
Epoch 21, Loss: 491212.59375
Epoch 22, Loss: 1560848.125
Epoch 23, Loss: 464143.40625
Epoch 24, Loss: 430557.71875
Epoch 25, Loss: 331889.59375
Epoch 26, Loss: 343707.34375
Epoch 27, Loss: 13451.72265625
Epoch 28, Loss: 67671.0859375
Epoch 29, Loss: 23286.591796875
Epoch 30, Loss: 12850.04296875
Epoch 31, Loss: 14345.3583984375
Epoch 32, Loss: 70476.4609375
Epoch 33, Loss: 14510.6181640625
Epoch 34, Loss: 12591.5986328125
Epoch 35, Loss: 71362.1953125
Epoch 36, Lo

In [61]:
model.eval()  # Set the model to evaluation mode
predictions = []
targets = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        predictions.extend(outputs.view(-1).tolist())  # Flatten outputs to list
        targets.extend(labels.view(-1).tolist())  # Flatten labels to list

predictions = np.array(predictions)
targets = np.array(targets)

rmse = mean_squared_error(targets, predictions, squared=False)
r2 = r2_score(targets, predictions)

print(f'RMSE: {rmse:.4f}')
print(f'R^2 Score: {r2:.4f}')

RMSE: 141.5611
R^2 Score: 0.9745


/Users/joshuaevans/opt/anaconda3/envs/2024_env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


## Make predictions based on data in the Master DF

In [62]:
X = preprocessor.transform(complete_merged_df.drop(['TOTALDEMAND'], axis=1))

X_tensor = torch.tensor(X.astype(np.float32)).to(device)

In [63]:
model.eval()  # Ensure the model is in evaluation mode
with torch.no_grad():
    predictions = model(X_tensor)
    predictions = predictions.view(-1).cpu().numpy()  

In [64]:
complete_merged_df['LSTM_PREDICTED_DEMAND'] = predictions


In [65]:

complete_merged_df['Error_LSTM'] = complete_merged_df['LSTM_PREDICTED_DEMAND'] - complete_merged_df['TOTALDEMAND']

complete_merged_df['Error_Ratio_LSTM'] = complete_merged_df['Error_LSTM'] / complete_merged_df['LSTM_PREDICTED_DEMAND']

print(complete_merged_df[['TOTALDEMAND', 'LSTM_PREDICTED_DEMAND', 'Error_LSTM', 'Error_Ratio_LSTM']].head())

   TOTALDEMAND  LSTM_PREDICTED_DEMAND  Error_LSTM  Error_Ratio_LSTM
0      5737.03            5898.669922  161.639922          0.027403
1      5897.64            6092.027344  194.387344          0.031908
2      6144.16            6264.055664  119.895664          0.019140
3      6264.63            6381.062500  116.432500          0.018247
4      6443.62            6580.881348  137.261348          0.020858


In [67]:
complete_merged_df[['Error_LSTM','Error_Ratio_LSTM']].describe()

,Error_LSTM,Error_Ratio_LSTM
count,53135.000000,53135.000000
mean,-23.424650,-0.003660
std,128.513832,0.019748
min,-1385.848906,-0.227482
25%,-92.131846,-0.015190
50%,-20.021602,-0.003346
75%,49.859570,0.008334
max,1142.821953,0.135490


## Attemping to predict demand for yesterday

In [87]:
import numpy as np
import pandas as pd
import torch

data = {
    'YEAR': [2024],
    'MONTH': [4],
    'DAY': [13],
    'HOUR': [13],
    'MINUTE': [0],
    'TEMPERATURE': [26],
    'Heating': [0],
    'Cooling': [2],
    'Solar_Production': [3660],
    'season': ['Autumn'],
    'is_weekend': [True],
    'Public_Holiday': [False]
}

df = pd.DataFrame(data)

df_processed = preprocessor.transform(df)

df_tensor = torch.tensor(df_processed.astype(np.float32))

print("Original tensor shape:", df_tensor.shape)

df_tensor = df_tensor.view(1, -1)  # Shape: [1, number_of_features]

model.eval()

with torch.no_grad():
    prediction = model(df_tensor)  # Pass the tensor, and let the model handle the reshaping

print("Predicted total demand at 1 PM yesterday:", prediction.item())


Original tensor shape: torch.Size([1, 15])
Predicted total demand at 1 PM yesterday: 5219.572265625


## Prediction Demand for the Past Week

In [88]:
data = {
    'YEAR': [2024, 2024, 2024, 2024, 2024, 2024, 2024],
    'MONTH': [4, 4, 4, 4, 4, 4, 4],
    'DAY': [8, 9, 10, 11, 12, 13, 14],
    'HOUR': [12, 12, 12, 12, 12, 12, 12],
    'MINUTE': [0, 0, 0, 0, 0, 0, 0],
    'TEMPERATURE': [28, 27, 25, 27, 27, 26, 27],
    'Heating': [0, 0, 0, 0, 0, 0, 0],
    'Cooling': [4, 3, 1, 3, 3, 2, 3],
    'Solar_Production': [3523, 3141, 3854, 3738, 3437, 3218, 3256],
    'season': ['Autumn', 'Autumn', 'Autumn', 'Autumn', 'Autumn', 'Autumn', 'Autumn'],
    'is_weekend': [False, False, False, False, True, True, True],
    'Public_Holiday': [False, False, False, False, False, False, False]
}

df = pd.DataFrame(data)

df_processed = preprocessor.transform(df)

df_tensor = torch.tensor(df_processed.astype(np.float32))

model.eval()

predictions = []

with torch.no_grad():
    for i in range(df_tensor.shape[0]):
        single_input = df_tensor[i].unsqueeze(0)  # Add batch dimension
        prediction = model(single_input)  # The model adds the sequence dimension internally
        predictions.append(prediction.item())

for i, pred in enumerate(predictions, start=1):
    print(f"Predicted total demand for {data['DAY'][i-1]}/{data['MONTH'][i-1]}/{data['YEAR'][i-1]} at 12 PM: {pred}")



Predicted total demand for 8/4/2024 at 12 PM: 5945.32861328125
Predicted total demand for 9/4/2024 at 12 PM: 5996.8271484375
Predicted total demand for 10/4/2024 at 12 PM: 5920.494140625
Predicted total demand for 11/4/2024 at 12 PM: 5813.892578125
Predicted total demand for 12/4/2024 at 12 PM: 5238.0546875
Predicted total demand for 13/4/2024 at 12 PM: 5290.056640625
Predicted total demand for 14/4/2024 at 12 PM: 5369.7021484375


## Predicted Demand for the forward week

In [91]:
new_data = {
    'YEAR': [2024] * 7,
    'MONTH': [4] * 7,
    'DAY': [15, 16, 17, 18, 19, 20, 21],
    'HOUR': [12] * 7,
    'MINUTE': [0] * 7,
    'TEMPERATURE': [27, 28, 27, 27, 27, 25, 25],
    'Heating': [0, 0, 0, 0, 0, 0, 0],
    'Cooling': [3, 4, 3, 3, 3, 1, 1],  # Corrected 'Cooling' values for the last two days
    'Solar_Production': [2947, 3153, 2814, 2859, 2897, 2590, 2702],
    'season': ['Autumn'] * 7,
    'is_weekend': [False, False, False, False, False, True, True],
    'Public_Holiday': [False, False, False, False, False, False, False]
}

new_df = pd.DataFrame(new_data)

new_df_processed = preprocessor.transform(new_df)

new_df_tensor = torch.tensor(new_df_processed.astype(np.float32))

model.eval()

new_predictions = []

with torch.no_grad():
    for i in range(new_df_tensor.shape[0]):
        single_input = new_df_tensor[i].unsqueeze(0)  # Add batch dimension
        prediction = model(single_input)  # The model adds the sequence dimension internally
        new_predictions.append(prediction.item())

for i, pred in enumerate(new_predictions, start=1):
    print(f"Predicted total demand for {new_data['DAY'][i-1]}/{new_data['MONTH'][i-1]}/{new_data['YEAR'][i-1]} at 12 PM: {pred}")


Predicted total demand for 15/4/2024 at 12 PM: 6010.04541015625
Predicted total demand for 16/4/2024 at 12 PM: 6017.76904296875
Predicted total demand for 17/4/2024 at 12 PM: 5993.50390625
Predicted total demand for 18/4/2024 at 12 PM: 5935.2939453125
Predicted total demand for 19/4/2024 at 12 PM: 5862.8291015625
Predicted total demand for 20/4/2024 at 12 PM: 5780.4521484375
Predicted total demand for 21/4/2024 at 12 PM: 5690.97802734375
